In [ ]:
!pip install weaviate-client langchain pypdf

In [ ]:
!pip install -U langchain-community

using weaviate as vector database on cloud.

In [ ]:
from google.colab import userdata
API = userdata.get('WEAVIATE_API_KEY')

In [ ]:
WEAVIATE_CLUSTER = "https://b0cjes5t72wfbqnr3t1ua.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY = API

In [ ]:
import weaviate
from weaviate.classes.init import Auth

# Best practice: store your credentials in environment variables
weaviate_url = WEAVIATE_CLUSTER
weaviate_api_key = WEAVIATE_API_KEY

client = weaviate.connect_to_weaviate_cloud(
    cluster_url = weaviate_url,  # Replace with your Weaviate Cloud URL
    auth_credentials = Auth.api_key(weaviate_api_key),  # Replace with your Weaviate Cloud key
)

In [ ]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [ ]:
!pip install sentence-transformers

In [ ]:
# specify embedding model (using huggingface sentence transformer)
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model_name = "sentence-transformers/all-mpnet-base-v2"

#model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  #model_kwargs=model_kwargs
)

<ipython-input-8-c10546a5fc95>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# let's check my langchain version, just time pas
import langchain
langchain.__version__

'0.3.13'

In [ ]:
!pip install rapidocr-onnxruntime  # ye pdf loader me kam aata hai

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/eqtl_2_pages.pdf", extract_images=True)
pages = loader.load()

# be cautious, 15 page pdf me sara 12 gb ran use ho gya tha.. So just keval 2 pages ka pdf loading

In [ ]:
pages

[Document(metadata={'source': '/content/eqtl_2_pages.pdf', 'page': 0}, page_content='eQTLs play critical roles in regulating gene expression and\nidentifying key regulators in rice\nChang Liu1, Xiya Zhu1, Jin Zhang1, Meng Shen1, Kai Chen1, Xiangkui Fu1, Lian Ma1, Xuelin Liu1, Chang Zhou1,\nDao-Xiu Zhou1,2 and Gongwei Wang1,*\n1National Key Laboratory of Crop Genetic Improvement, Hubei Hongshan Laboratory, Huazhong Agricultural University, Wuhan, China\n2Institute of Plant Science Paris-Saclay (IPS2), CNRS, INRAE, University Paris-Saclay, Orsay, France\nReceived 17 June 2022;\nrevised 11 August 2022;\naccepted 13 August 2022.\n*Correspondence (Tel +86-15827398206;\nfax +86-27-87280916; email\ngwwang@mail.hzau.edu.cn)\nKeywords: population RNA-seq,\nexpression quantitative trait loci\n(eQTLs), diterpenoid antitoxin, key\nregulators, transcriptome-wide\nassociation study (TWAS).\nSummary\nThe regulation of gene expression plays an essential role in both the phenotype and adaptation\nof pl

In [ ]:
# Split text into chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)

In [ ]:
docs

[Document(metadata={'source': '/content/eqtl_2_pages.pdf', 'page': 0}, page_content='eQTLs play critical roles in regulating gene expression and\nidentifying key regulators in rice\nChang Liu1, Xiya Zhu1, Jin Zhang1, Meng Shen1, Kai Chen1, Xiangkui Fu1, Lian Ma1, Xuelin Liu1, Chang Zhou1,\nDao-Xiu Zhou1,2 and Gongwei Wang1,*\n1National Key Laboratory of Crop Genetic Improvement, Hubei Hongshan Laboratory, Huazhong Agricultural University, Wuhan, China\n2Institute of Plant Science Paris-Saclay (IPS2), CNRS, INRAE, University Paris-Saclay, Orsay, France\nReceived 17 June 2022;\nrevised 11 August 2022;\naccepted 13 August 2022.\n*Correspondence (Tel +86-15827398206;\nfax +86-27-87280916; email\ngwwang@mail.hzau.edu.cn)\nKeywords: population RNA-seq,\nexpression quantitative trait loci\n(eQTLs), diterpenoid antitoxin, key\nregulators, transcriptome-wide\nassociation study (TWAS).\nSummary\nThe regulation of gene expression plays an essential role in both the phenotype and adaptation\nof pl

In [ ]:
len(docs)

# total 15 chunks formed because of recursivetextsplitter

15

In [ ]:
print(client.is_ready())

True


In [ ]:
!pip install langchain_weaviate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 8.8 MB/s eta 0:00:00


In [ ]:
from langchain_weaviate.vectorstores import WeaviateVectorStore

db = WeaviateVectorStore.from_documents(docs, embeddings, client = client)


In [ ]:
db

In [ ]:
db.similarity_search("what is GWAS?", k=3)

[Document(metadata={'page': 0.0, 'source': '/content/eqtl_2_pages.pdf'}, page_content='major model crop for genetic improvement. Large amounts of\nhigh-density genotype data and related high-throughput pheno-\ntypic data of rice have been accumulated and associated with\nmany important candidate genes through GWAS approaches\n(Huang et al., 2012; Wang et al., 2015). GWAS can identify\ncandidate associations by detecting variations across genotypes\nand phenotypes, but cannot accurately identify candidate genes\nand gene functions due to the decay of linkage disequilibrium\nand limited gene annotation. In recent studies, some researchers\nhave used methods such as transcriptome-wide association study\n(TWAS) or Camoco that can combine genomic and transcriptomic\ndata (Gusev et al., 2016; Schaefer et al., 2018) to more accu-\nrately predict the candidate genes for phenotypic variations (Tang\net al., 2021; Walker et al., 2019). eQTLs and co-localized QTLs\nare equally important for disse

In [ ]:
db.similarity_search("what is GWAS?", k=3)[0]

Document(metadata={'page': 0.0, 'source': '/content/eqtl_2_pages.pdf'}, page_content='major model crop for genetic improvement. Large amounts of\nhigh-density genotype data and related high-throughput pheno-\ntypic data of rice have been accumulated and associated with\nmany important candidate genes through GWAS approaches\n(Huang et al., 2012; Wang et al., 2015). GWAS can identify\ncandidate associations by detecting variations across genotypes\nand phenotypes, but cannot accurately identify candidate genes\nand gene functions due to the decay of linkage disequilibrium\nand limited gene annotation. In recent studies, some researchers\nhave used methods such as transcriptome-wide association study\n(TWAS) or Camoco that can combine genomic and transcriptomic\ndata (Gusev et al., 2016; Schaefer et al., 2018) to more accu-\nrately predict the candidate genes for phenotypic variations (Tang\net al., 2021; Walker et al., 2019). eQTLs and co-localized QTLs\nare equally important for dissec

In [ ]:
print(db.similarity_search("what is GWAS?", k=3)[0].page_content)

major model crop for genetic improvement. Large amounts of
high-density genotype data and related high-throughput pheno-
typic data of rice have been accumulated and associated with
many important candidate genes through GWAS approaches
(Huang et al., 2012; Wang et al., 2015). GWAS can identify
candidate associations by detecting variations across genotypes
and phenotypes, but cannot accurately identify candidate genes
and gene functions due to the decay of linkage disequilibrium
and limited gene annotation. In recent studies, some researchers
have used methods such as transcriptome-wide association study
(TWAS) or Camoco that can combine genomic and transcriptomic
data (Gusev et al., 2016; Schaefer et al., 2018) to more accu-
rately predict the candidate genes for phenotypic variations (Tang
et al., 2021; Walker et al., 2019). eQTLs and co-localized QTLs
are equally important for dissecting the genetic architecture of


In [ ]:
print(db.similarity_search("what is GWAS?", k=3)[1].page_content)

in leaves at the heading stage, accounting for about 41.8% of
the total annotated genes (55 801) in the MSU 7.0 genome.
Using the Fast-LMM software (Lippert et al., 2011), with
genomic SNP data from 287 accessions, the transformed
expression levels of each gene were used for association analysis
with the SNPs in the genome. As a result, the expression of
14 562 genes was signiﬁcantly associated with at least one SNP
over the Bonferroni-corrected threshold (P = 5.43 9 10/C08;
a = 0.05). SNPs associated with the same gene were clustered
into one unique eQTL block including at least three SNPs, and
the SNP with the lowestP-value was used to represent this
block. A total of 44 354 eQTLs were identiﬁed from 13 201
genes (Table S3).
According to the relative positions of genes and their corre-
sponding eQTLs in the genome, a strong diagonal enrichment
could be observed (Figure2a). In addition, based on the relative
distance between eQTLs and genes, all eQTLs could be divided


In [ ]:

from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
huggingfacehub_api_token = userdata.get('HUGGINGFACE_API_KEY')

In [ ]:
# huggingfacehub_api_token

In [ ]:
# access huggingface model without downloading them locally, through api.

model = HuggingFaceHub(
    huggingfacehub_api_token = huggingfacehub_api_token,
    repo_id = "mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":1, "max_length":180}
)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
output_parser = StrOutputParser()

In [ ]:
retriever = db.as_retriever()

In [ ]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [ ]:
rag_chain.invoke("what is Summary of the paper")

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1 (Request ID: DDn_PIJCaDFy1J2XHL_Hx)

Rate limit reached. You reached free usage limit (reset daily). Please subscribe to PRO or Enterprise Hub to get a higher limit: https://hf.co/pricing

Noice, huggingface api ki limit khatam hi gyi, nhi t vhi pr direct answer aa jata